<a href="https://colab.research.google.com/github/mohameddhameem/NLPWithPyTorch/blob/master/Learn_BERT_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Install the additional libaries.
! pip install -U git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-vaqy5exi
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-vaqy5exi
     |████████████████████████████████| 3.7MB 2.5MB/s 
     |████████████████████████████████| 1.0MB 16.9MB/s 
     |████████████████████████████████| 870kB 33.7MB/s 
  Created wheel for transformers: filename=transformers-2.5.1-cp36-none-any.whl size=534520 sha256=0970ae3a0084a573b1e341fdcd6247b0be6d0a18dbd935959b43d239003f05de
  Stored in directory: /tmp/pip-ephem-wheel-cache-efiihmb7/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=195bc3a6a126acde834ba6f337f87723686e1370b7ee9f4d5bad0460b361a02f
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built transformers sacremoses


In [3]:
from itertools import chain
from collections import namedtuple

import numpy as np
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# Load pre-trained model tokenizer (vocabulary)
# A tokenizer will split the text into the appropriate sub-parts (aka. tokens).
# Depending on how the pre-trained model is trained, the tokenizers defers.
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# Example of a tokenized input after WordPiece Tokenization.
text = "[CLS] my dog is cute [SEP] he likes playing [SEP]"
print(tokenizer.wordpiece_tokenizer.tokenize(text))


['[CLS]', 'my', 'dog', 'is', 'cute', '[SEP]', 'he', 'likes', 'playing', '[SEP]']


In [6]:
"playing" in tokenizer.wordpiece_tokenizer.vocab

True

In [7]:
print("slacking" in tokenizer.wordpiece_tokenizer.vocab)

False


In [8]:
text = "[CLS] my dog is cute [SEP] he likes slacking [SEP]"
tokenized_text = tokenizer.wordpiece_tokenizer.tokenize(text) # There, we see the ##ing token!
print(tokenized_text)

['[CLS]', 'my', 'dog', 'is', 'cute', '[SEP]', 'he', 'likes', 'slack', '##ing', '[SEP]']


In [9]:
token_indices = tokenizer.convert_tokens_to_ids(tokenized_text)
token_indices

[101, 2026, 3899, 2003, 10140, 102, 2002, 7777, 19840, 2075, 102]

In [10]:
import numpy as np

# We need to create an array that indicates the end of sentences, delimited by [SEP]
text = "[CLS] my dog is cute [SEP] he likes slacking [SEP]"
tokenized_text = tokenizer.wordpiece_tokenizer.tokenize(text)  # There, we see the ##ing token!

# First we find the indices of `[SEP]`, and incrementally adds it up. 
# Here's some Numpy gymnastics... 
# Thanks to @divakar https://stackoverflow.com/a/58316889/610569
m = np.asarray(tokenized_text) == "[SEP]"
segments_ids = m.cumsum()-m

tokens_tensor, segments_tensors = torch.tensor([token_indices]), torch.tensor([segments_ids])

# See the type change?
print(tokens_tensor.shape, type(token_indices), type(tokens_tensor))
print(segments_tensors.shape, type(segments_ids), type(segments_tensors))

torch.Size([1, 11]) <class 'list'> <class 'torch.Tensor'>
torch.Size([1, 11]) <class 'numpy.ndarray'> <class 'torch.Tensor'>


In [11]:
tokens_tensor, segments_tensors = torch.tensor([token_indices]), torch.tensor([segments_ids])

# See the type change?
print(tokens_tensor.shape, type(token_indices), type(tokens_tensor))
print(segments_tensors.shape, type(segments_ids), type(segments_tensors))

torch.Size([1, 11]) <class 'list'> <class 'torch.Tensor'>
torch.Size([1, 11]) <class 'numpy.ndarray'> <class 'torch.Tensor'>


# Lets convert our input text to an array of number

## First, we load the pre-trained model

In [12]:
import torch 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# When using the BERT model for "encoding", i.e. convert string to array of floats, 
# we use the `BertModel` object from pytorch transformer library.
model = BertModel.from_pretrained('bert-base-uncased')
model.eval(); model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [13]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor.to(device), segments_tensors.to(device))
    
print(encoded_layers)

tensor([[[ 0.0737,  0.1229, -0.1612,  ...,  0.1235,  0.2381,  0.0880],
         [ 0.5442,  0.6884, -0.0744,  ..., -0.0346,  0.1388, -0.4474],
         [ 0.7117,  0.6298, -0.0084,  ...,  0.1541,  0.1260, -0.1969],
         ...,
         [ 0.6583,  0.4991, -0.1022,  ...,  0.2715, -0.2863, -0.4839],
         [-0.1440, -0.3067, -0.3331,  ...,  0.4313,  0.1080,  0.0214],
         [ 0.9325,  0.2506,  0.0805,  ...,  0.1434, -0.8708, -0.4659]]],
       device='cuda:0')


In [14]:
encoded_layers.shape

torch.Size([1, 11, 768])

# The BERT model is very good at fill-in-the-blank task

The BERT model is trained using a "cloze" task where words are randomly replaced with the [MASK] symbols and the model learns to adjust its parameters such that it learns which words are most probable to fit into the [MASK] symbols.

When using the BERT model for "guessing missing words", we use the BertForMaskedLM object from pytorch transformer library. Here's an example if we blank out words in the sentence, BERT is able to find the appropriate word to fill it in.

In [15]:
# Load the model.
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval(); model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [16]:
# We need to create an array that indicates the end of sentences, delimited by [SEP]
text = "[CLS] please don't let the [MASK] out of the [MASK] . [SEP]"
tokenized_text = tokenizer.wordpiece_tokenizer.tokenize(text)
token_indices = tokenizer.convert_tokens_to_ids(tokenized_text)

# Create the segment indices.
m = np.asarray(tokenized_text) == "[SEP]"
segments_ids = m.cumsum()-m

# Convert them to the arrays to pytorch tensors.
tokens_tensor, segments_tensors = torch.tensor([token_indices]), torch.tensor([segments_ids])

# Apply the model to the inputs.
with torch.no_grad(): # You can take this context manager to mean that we're not training.
    outputs, *_ = model(tokens_tensor.to(device), 
                        token_type_ids=segments_tensors.to(device))

outputs.shape

torch.Size([1, 14, 30522])

In [0]:
# Apply the model to the inputs.
with torch.no_grad(): # You can take this context manager to mean that we're not training.
    outputs, *_ = model(tokens_tensor.to(device), token_type_ids=segments_tensors.to(device))

In [18]:
outputs.shape

torch.Size([1, 14, 30522])

In [19]:
print(tokenized_text)

['[CLS]', 'please', 'don', "##'", '##t', 'let', 'the', '[MASK]', 'out', 'of', 'the', '[MASK]', '.', '[SEP]']


In [20]:
# Lets remember our original masked sentence.
print(tokenized_text)
# We have to check where the masked token is from the original text. 
mask_index = tokenized_text.index('[MASK]') 
assert mask_index == 7 # The 7th token.

# Then we fetch the vector for the 7th value, 
# The [0, mask_index] refers to accessing vector of vocab_size for
# the 0th sentence, mask_index-th token.
output_value = outputs[0, mask_index]

# As a sanity check we can see that the shape of the output_value
# is the same as the `vocab_size` from the outputs' shape.
assert int(output_value.shape[0]) == len(tokenizer.wordpiece_tokenizer.vocab)

['[CLS]', 'please', 'don', "##'", '##t', 'let', 'the', '[MASK]', 'out', 'of', 'the', '[MASK]', '.', '[SEP]']


In [21]:
# Lets recap the original sentence with the masked word.
print(text)

# We have to check where the first masked token is from the original text. 
mask_index = tokenized_text.index('[MASK]') 
output_value = outputs[0, mask_index]

## We use torch.argmax to get the index with the highest value.
mask_word_in_vocab = int(torch.argmax(output_value))
print(tokenizer.convert_ids_to_tokens([mask_word_in_vocab]))

[CLS] please don't let the [MASK] out of the [MASK] . [SEP]
['cat']


In [22]:
# Lets recap the original sentence with the masked word.
print(text)

# We have to check where the masked tokens are from the original text. 
for mask_index, token in enumerate(tokenized_text):
    if token == '[MASK]':
        output_value = outputs[0, mask_index]
        mask_word_in_vocab = int(torch.argmax(output_value))
        print(tokenizer.convert_ids_to_tokens([mask_word_in_vocab]))

[CLS] please don't let the [MASK] out of the [MASK] . [SEP]
['cat']
['way']


# Lets make the fill-in-the-blank feature into a function.

In [0]:
def fill_in_the_blanks(text, model, tokenizer, return_str=False):
    tokenized_text = tokenizer.wordpiece_tokenizer.tokenize(text)
    token_indices = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Create the segment indices.
    m = np.asarray(tokenized_text) == "[SEP]"
    segments_ids = m.cumsum()-m
    # Convert them to the arrays to pytorch tensors.
    tokens_tensor = torch.tensor([token_indices]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
    
    # Apply the model to the inputs.
    with torch.no_grad(): # You can take this context manager to mean that we're not training.
        outputs, *_ = model(tokens_tensor, token_type_ids=segments_tensors)
    
    output_tokens = []
    for mask_index, token_id in enumerate(token_indices):
        token = tokenizer.convert_ids_to_tokens([token_id])[0]
        if token == '[MASK]':
            output_value = outputs[0, mask_index]
            # The masked word index in the vocab.
            mask_word_in_vocab = int(torch.argmax(output_value))
            token = tokenizer.convert_ids_to_tokens([mask_word_in_vocab])[0]
        output_tokens.append(token)
        
    return " ".join(output_tokens).replace(" ##", "").replace(" ' t ", "'t ") if return_str else output_tokens

In [24]:
# Load the model.
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval(); model.to(device)

text = "[CLS] please don't let the [MASK] out of the [MASK] . [SEP]"
print(fill_in_the_blanks(text, model, tokenizer, return_str=True))

[CLS] please don't let the cat out of the way . [SEP]


In [25]:
text = "[CLS] i like to drink beer and eat [MASK] . [SEP]"
print(fill_in_the_blanks(text, model,tokenizer, return_str=True))

[CLS] i like to drink beer and eat meat . [SEP]


In [26]:
text = "[CLS] i like to drink coffee and eat [MASK] . [SEP]"
print(fill_in_the_blanks(text, model, tokenizer, return_str=True))

[CLS] i like to drink coffee and eat it . [SEP]
